In [1]:
%load_ext lab_black

In [2]:
import os

os.chdir("/cerea_raid/users/dumontj/dev/coco2/dl")
import numpy as np
import sys
import pandas as pd
import xarray as xr
import tensorflow as tf
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib_functions as mympf

mympf.setMatplotlibParam()
plt.viridis()
import joblib
import pickle
from include.generators import Generator, ScaleDataGen
from saver import Saver
from include.callbacks import get_modelcheckpoint, ExtraValidation
from keras.utils.vis_utils import plot_model
import model_eval
from Data import Data_train, Data_eval
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger
import models.reg as rm
import tensorflow as tf
from dataclasses import dataclass, field


from keras import backend as K

2023-02-28 15:05:40.147893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<Figure size 1920x1440 with 0 Axes>

In [3]:
path_train_nc = "/libre/dumontj/coco2/dl-input/2km_forLip_2/train_dataset.nc"
path_valid_nc = "/libre/dumontj/coco2/dl-input/2km_forLip_2/valid_dataset.nc"
path_extra_valid_nc = "/libre/dumontj/coco2/dl-input/2km_Lip/valid_dataset.nc"

In [4]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("xco2", "u_wind", "v_wind", "xco2_prec", "bool_perf_seg")
data.prepare_output_inversion()

2023-02-28 15:05:52.344493: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 15:05:52.852947: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14482 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5
2023-02-28 15:05:53.026365: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2060451840 exceeds 10% of free system memory.
2023-02-28 15:05:54.312745: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2060451840 exceeds 10% of free system memory.


data.x.train.shape (25152, 64, 64, 5)


In [5]:
generator = ScaleDataGen(
    data.x.train,
    data.x.plumes_train,
    data.y.train,
    data.x.scale_bool,
    data.x.fields_input_shape,
)

In [6]:
generator = Generator("inversion")
generator_flowing = generator.flow(data.x.train, data.y.train)

/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (25152, 64, 64, 5) (5 channels).
  warnings.warn(


In [7]:
reg_builder = rm.Reg_model_builder(
    "essential",
    data.x.fields_input_shape,
    data.y.classes,
    data.x.n_layer,
    data.x.xco2_noisy_chans,
)
model = reg_builder.get_model()
model.compile("adam", "MeanAbsoluteError")
model.fit(generator_flowing, epochs=15)

Epoch 1/15


2023-02-28 15:05:59.036891: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/model/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-28 15:05:59.796745: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-28 15:06:00.366781: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-28 15:06:00.716388: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fcddc01b940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-28 15:06:00.716436: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2023-02-28 15:06:00.726986: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set 

786/786 [==============================] - 16s 14ms/step - loss: 5.8722
Epoch 2/15
786/786 [==============================] - 11s 14ms/step - loss: 4.3527
Epoch 3/15
786/786 [==============================] - 11s 14ms/step - loss: 3.7790
Epoch 4/15
786/786 [==============================] - 11s 14ms/step - loss: 3.4419
Epoch 5/15
786/786 [==============================] - 11s 14ms/step - loss: 3.1799
Epoch 6/15
786/786 [==============================] - 11s 14ms/step - loss: 3.0636
Epoch 7/15
786/786 [==============================] - 11s 14ms/step - loss: 2.9310
Epoch 8/15
786/786 [==============================] - 11s 14ms/step - loss: 2.9755
Epoch 9/15
786/786 [==============================] - 11s 14ms/step - loss: 2.8533
Epoch 10/15
786/786 [==============================] - 11s 14ms/step - loss: 2.7801
Epoch 11/15
786/786 [==============================] - 11s 14ms/step - loss: 2.7189
Epoch 12/15
786/786 [==============================] - 11s 14ms/step - loss: 2.7178
Epoch 13/15


In [9]:
model.fit(
    generator_flowing,
    epochs=15,
    validation_data=(data.x.extra_valid, data.y.extra_valid),
)

Epoch 1/15
786/786 [==============================] - 11s 14ms/step - loss: 2.3980 - val_loss: 5.3827
Epoch 2/15
786/786 [==============================] - 11s 14ms/step - loss: 2.3960 - val_loss: 5.7426
Epoch 3/15
786/786 [==============================] - 11s 14ms/step - loss: 2.3867 - val_loss: 5.2785
Epoch 4/15
786/786 [==============================] - 11s 14ms/step - loss: 2.3610 - val_loss: 5.2396
Epoch 5/15
185/786 [======>.......................] - ETA: 8s - loss: 2.4158


KeyboardInterrupt

